In [7]:
from custom_dataset import custom_collate, Dataset_MutationList, Dataset_TopN_emb
from models import *

# LOAD DATA
# canonical_mut_embeddings_esm2 = np.load('../aa/canonical_mut_embeddings_esm2.npy')
canonical_ave_embeddings_esm2 = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)
data = labeled_data[17]
print('\n',data)
data_df = pd.read_csv(data_dir+data)
nlabels = len(data_df['int_label'].unique())
device = 'cuda:1'

# Create dataset
# dataset = Dataset_MutationList(data_df, canonical_mut_embeddings_esm2,device)
dataset = Dataset_MutationList(data_df, canonical_ave_embeddings_esm2,device)

# Create DataLoader
# dataloader = DataLoader(dataset, batch_size=100, shuffle=False, collate_fn=custom_collate)

## TEST/TRAIN SPLIT
test_size = .2
random_state = 42
batch_size = 1
indices = list(range(len(dataset)))

train_indices, test_indices = train_test_split(
    indices, 
    test_size=test_size, 
    random_state=random_state
)

train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

37 	 BINARYdata_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
24 	 BINARYdata_CANCER_TYPE_0MinMutations_169MinCancerType.csv
1 	 BINARYdata_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
32 	 BINARYdata_CANCER_TYPE_3MinMutations_169MinCancerType.csv
12 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
26 	 BINARYdata_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
10 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
30 	 BINARYdata_CANCER_TYPE_DETAILED_3MinMutations_169MinCancerType.csv
23 	 data_CANCER_TYPE_0MinMutations_1696MinCancerType.csv
25 	 data_CANCER_TYPE_0MinMutations_169MinCancerType.csv
41 	 data_CANCER_TYPE_3MinMutations_1696MinCancerType.csv
17 	 data_CANCER_TYPE_3MinMutations_169MinCancerType.csv
11 	 data_CANCER_TYPE_DETAILED_0MinMutations_1696MinCancerType.csv
28 	 data_CANCER_TYPE_DETAILED_0MinMutations_169MinCancerType.csv
4 	 data_CANCER_TYPE_DETAILED_3MinMutations_1696MinCancerType.csv
47 	 data_CANCER_TYPE_DETAIL

In [8]:
# majority classifier
print(len(data_df['int_label'].unique()))
sorted(data_df['int_label'].value_counts(),reverse=True)[0]/len(data_df['int_label'])

53


0.16158075053780574

In [9]:
import torch.optim as optim
from tqdm import tqdm

class Config:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 17
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    position_embedding: bool = False
    max_len: int = 30

print('n labels:',nlabels)
config = Config()
config.n_labels = nlabels

model = Classifier(config)
model.to(device)

num_epochs = 100
learning_rate = 0.001

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    with tqdm(enumerate(train_loader), total=len(train_loader),desc='TRAINING') as pbar:
        for batch_idx, (data, target) in pbar:
            optimizer.zero_grad()
            output = model(data)
            # assert False
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            pbar.set_postfix({'Epoch':f'{epoch+1}/{num_epochs}, Loss: {loss.item():.4f}'})
            if batch_idx % 20000 == 0:
                print('')

        # Evaluation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for data, target in tqdm(test_loader,desc='TESTING'):
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%, ({correct} of {total})')

n labels: 53


TRAINING:   0%|          | 0/90367 [00:00<?, ?it/s, Epoch=1/100, Loss: 4.4586]

TRAINING:  22%|██▏       | 20015/90367 [03:40<13:25, 87.29it/s, Epoch=1/100, Loss: 7.7460]   

TRAINING:  22%|██▏       | 20250/90367 [03:43<13:32, 86.28it/s, Epoch=1/100, Loss: 0.2537] 